In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import arcpy
import arcgisscripting
import glob
import os
import rasterio as rio
import rasterio.mask
import rasterio.plot as rio_pl
import matplotlib.pyplot as plt
from arcpy.sa import *
from arcpy import env
from simpledbf import Dbf5
arcpy.env.overwriteOutput = True

PyTables is not installed. No support for HDF output.


# 1 Calculate percentile and reclassify the maps

In [88]:
path = r"F:\Water_Quality\KDE_SEP_TIF_All\0629"
all_files = glob.glob(os.path.join(str(path+"/OriginalRaster") , "*.tif"))
zone_files= r"D:\Water_Quailty\Managed_Areas.shp"
SpatialRef= "EPSG:3086"
env.workspace = path
in_zone = gpd.read_file(zone_files,crs=SpatialRef)

In [93]:
for i in range(len(all_files)):
    full_name = all_files[i].split("\\")[-1]
    name = full_name.split(".")[0]
    output_path = r"F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/"
    new_name = output_path+name+"_Resample.tif"
    print(new_name)
    arcpy.Resample_management(all_files[i],new_name, "10 10", "NEAREST")

F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBAP_DO_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBAP_Sal_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBAP_Secchi_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBAP_TN_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBAP_Turb_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBSAP_DO_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBSAP_Sal_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBSAP_Secchi_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBSAP_TN_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_BBSAP_Turb_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_EBAP_DO_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_EBAP_Sal_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Resample/KDE_EBAP_Secchi_Resample.tif
F:\Water_Quality\KDE_SEP_TIF_All\0629\Re

In [94]:
Resample_file = glob.glob(os.path.join(str(path+"/Resample") , "*.tif"))

In [98]:
for i in range(len(Resample_file)):
    outZSaT = ZonalStatisticsAsTable(in_zone_data = zone_files,zone_field="MA_Name",in_value_raster=Resample_file[i],out_table=path+"/table.dbf",
                                     statistics_type="PERCENTILE",percentile_values=[0,25,75,100],percentile_interpolation_type="AUTO_DETECT")
    table = Dbf5(path+"/table.dbf")
    table = table.to_dataframe()
    print(table)
    percentile0  = float(table.PCT0)
    percentile25 = float(table.PCT25)
    percentile75 = float(table.PCT75)
    percentile100= float(table.PCT100)
    name = Resample_file[i].split("\\")[-1]
    reName = name.split(".")[0][:-9]+"_Reclass."+name.split(".")[1]
    print(reName)
    out_raster = Reclassify(Resample_file[i], "VALUE",
            RemapRange([[percentile0,percentile25,1],[percentile25,percentile75,2],[percentile75,percentile100,3]]), "NODATA")
    out_raster.save(path+"/Reclassify/"+reName)

        MA_Name  ZONE_CODE      COUNT         AREA  PCT0     PCT25    PCT75  \
0  Biscayne Bay          1  2585107.0  258510700.0   0.0  0.709912  2.86053   

        PCT100  
0  1029.002075  
KDE_BBAP_DO_Reclass.tif
        MA_Name  ZONE_CODE      COUNT         AREA  PCT0     PCT25     PCT75  \
0  Biscayne Bay          1  2585107.0  258510700.0   0.0  0.154657  3.136654   

        PCT100  
0  1149.473755  
KDE_BBAP_Sal_Reclass.tif
        MA_Name  ZONE_CODE      COUNT         AREA  PCT0  PCT25     PCT75  \
0  Biscayne Bay          1  2585107.0  258510700.0   0.0    0.0  0.229451   

     PCT100  
0  5.176455  
KDE_BBAP_Secchi_Reclass.tif
        MA_Name  ZONE_CODE      COUNT         AREA  PCT0     PCT25     PCT75  \
0  Biscayne Bay          1  2585107.0  258510700.0   0.0  0.221511  1.360301   

      PCT100  
0  31.006214  
KDE_BBAP_TN_Reclass.tif
        MA_Name  ZONE_CODE      COUNT         AREA  PCT0     PCT25     PCT75  \
0  Biscayne Bay          1  2585107.0  258510700.0   0.0 

      MA_Name  ZONE_CODE     COUNT        AREA      PCT0     PCT25     PCT75  \
0  Estero Bay          1  547695.0  54769500.0  0.021848  0.063555  0.126068   

     PCT100  
0  0.952516  
SEP_EBAP_DO_Reclass.tif
      MA_Name  ZONE_CODE     COUNT        AREA      PCT0     PCT25     PCT75  \
0  Estero Bay          1  547695.0  54769500.0  0.003343  0.091799  0.176128   

     PCT100  
0  0.955875  
SEP_EBAP_Sal_Reclass.tif
      MA_Name  ZONE_CODE     COUNT        AREA      PCT0    PCT25     PCT75  \
0  Estero Bay          1  547695.0  54769500.0  0.021084  0.05811  0.111668   

     PCT100  
0  0.837005  
SEP_EBAP_TN_Reclass.tif
      MA_Name  ZONE_CODE     COUNT        AREA      PCT0     PCT25     PCT75  \
0  Estero Bay          1  547695.0  54769500.0  0.027882  0.098257  0.157002   

     PCT100  
0  0.741331  
SEP_EBAP_Turb_Reclass.tif
                             MA_Name  ZONE_CODE      COUNT         AREA  \
0  Gasparilla Sound-Charlotte Harbor          1  3397656.0  339765600.0 

In [99]:
KDE_files_reclassify = glob.glob(os.path.join(str(path+"\Reclassify") , "KDE_*.tif"))
SEP_files_reclassify = glob.glob(os.path.join(str(path+"\Reclassify") , "SEP_*.tif"))

In [102]:
for j in range(len(SEP_files_reclassify)):
    name = SEP_files_reclassify[j].split("\\")[-1].split("_")
    coName = name[1] + "_" + name[2] + "_" + "Co.tif"
    KDE = path + "/Reclassify/" + "KDE_" + name[1] + "_" + name[2] + "_" + name[3]
    print([SEP_files_reclassify[j],KDE])
    out_raster = Combine([SEP_files_reclassify[j],KDE]); 
    out_raster.save(path+"/Overlay/"+coName)

['F:\\Water_Quality\\KDE_SEP_TIF_All\\0629\\Reclassify\\SEP_BBAP_DO_Reclass.tif', 'F:\\Water_Quality\\KDE_SEP_TIF_All\\0629/Reclassify/KDE_BBAP_DO_Reclass.tif']
['F:\\Water_Quality\\KDE_SEP_TIF_All\\0629\\Reclassify\\SEP_BBAP_Sal_Reclass.tif', 'F:\\Water_Quality\\KDE_SEP_TIF_All\\0629/Reclassify/KDE_BBAP_Sal_Reclass.tif']
['F:\\Water_Quality\\KDE_SEP_TIF_All\\0629\\Reclassify\\SEP_BBAP_Secchi_Reclass.tif', 'F:\\Water_Quality\\KDE_SEP_TIF_All\\0629/Reclassify/KDE_BBAP_Secchi_Reclass.tif']
['F:\\Water_Quality\\KDE_SEP_TIF_All\\0629\\Reclassify\\SEP_BBAP_TN_Reclass.tif', 'F:\\Water_Quality\\KDE_SEP_TIF_All\\0629/Reclassify/KDE_BBAP_TN_Reclass.tif']
['F:\\Water_Quality\\KDE_SEP_TIF_All\\0629\\Reclassify\\SEP_BBAP_Turb_Reclass.tif', 'F:\\Water_Quality\\KDE_SEP_TIF_All\\0629/Reclassify/KDE_BBAP_Turb_Reclass.tif']
['F:\\Water_Quality\\KDE_SEP_TIF_All\\0629\\Reclassify\\SEP_BBSAP_DO_Reclass.tif', 'F:\\Water_Quality\\KDE_SEP_TIF_All\\0629/Reclassify/KDE_BBSAP_DO_Reclass.tif']
['F:\\Water_Qualit

# 2 Overlay rasters

# 4 Get the value in the overlaid raster to each sampling point

In [103]:
data = pd.read_csv(r"F:\Water_Quality\KDE_SEP_TIF_All\Points\new\all_data.csv")

In [104]:
data_gdf = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.Lat,data.Long),crs="EPSG:4326")

In [105]:
data_gdf = data_gdf.to_crs(SpatialRef)

In [106]:
data_gdf.to_file(r"F:\Water_Quality\KDE_SEP_TIF_All\Points\new\all.shp")

C:\Users\xiang11\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [107]:
gdf_all_points = gpd.read_file(r"F:\Water_Quality\KDE_SEP_TIF_All\Points\new\all_points\all_points.shp")

In [108]:
gdf_all_points

,AreaID,ManagedAre,ParameterN,ProgramID,ProgramNam,Type,ProgramLoc,Lat,Long,geometry
0,5,Big Bend Seagrasses Aquatic Preserve,Dissolved Oxygen,69,Fisheries-Independent Monitoring (FIM) Program,Discrete,CKM2015090908,29.364916,-83.201950,POINT (477349.775 595621.578)
1,5,Big Bend Seagrasses Aquatic Preserve,Dissolved Oxygen,69,Fisheries-Independent Monitoring (FIM) Program,Discrete,CKM2016100409,29.314783,-83.118783,POINT (485450.219 590109.925)
2,5,Big Bend Seagrasses Aquatic Preserve,Dissolved Oxygen,69,Fisheries-Independent Monitoring (FIM) Program,Discrete,CKM2015030505,29.365066,-83.202400,POINT (477306.052 595637.952)
3,5,Big Bend Seagrasses Aquatic Preserve,Dissolved Oxygen,69,Fisheries-Independent Monitoring (FIM) Program,Discrete,CKM2015070108,29.142983,-83.019650,POINT (495214.415 571106.075)
4,5,Big Bend Seagrasses Aquatic Preserve,Dissolved Oxygen,69,Fisheries-Independent Monitoring (FIM) Program,Discrete,CKM2017050805,29.277700,-83.112116,POINT (486126.341 585996.981)
...,...,...,...,...,...,...,...,...,...,...
22918,20,Guana Tolomato Matanzas National Estuarine Res...,Salinity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmpiwq,30.050857,-81.367465,POINT (653510.323 674231.938)
22919,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmpcwq,29.667071,-81.257403,POINT (665054.955 631867.561)
22920,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmpiwq,30.050857,-81.367465,POINT (653510.323 674231.938)
22921,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmsswq,29.868851,-81.307428,POINT (659732.033 654157.227)


In [109]:
list_ManageAre = list(gdf_all_points.ManagedAre.unique())
list_ParameterN= list(gdf_all_points.ParameterN.unique())

In [110]:
dictPara = {'Dissolved Oxygen':"DO",'Salinity':"Sal",'Secchi Depth':"Secchi","Total Nitrogen":"TN","Turbidity":"Turb"}
dictArea = {"Gasparilla Sound-Charlotte Harbor Aquatic Preserve":"GSCHAP",'Big Bend Seagrasses Aquatic Preserve':"BBSAP",
                'Guana Tolomato Matanzas National Estuarine Research Reserve':"GTMNERR",'Estero Bay Aquatic Preserve':"EBAP",
                'Biscayne Bay Aquatic Preserve':"BBAP"}

In [112]:
for i in list_ManageAre:
    for j in list_ParameterN:
        gdf = gdf_all_points[(gdf_all_points["ManagedAre"] == i) & (gdf_all_points["ParameterN"] == j)]
        area= dictArea[i]
        para= dictPara[j]
        gdf.to_file(r"F:\Water_Quality\KDE_SEP_TIF_All\0629\Points\Separate/"+area+"_"+para+".shp")

In [122]:
Points  = glob.glob(os.path.join(str(r"F:\Water_Quality\KDE_SEP_TIF_All\0629\Points\Separate") , "*.shp"))
Rasters = glob.glob(os.path.join(str(path+"\Overlay") , "*.tif"))

In [125]:
for k in range(len(Points)):
    long_name = Points[k].split("\\")
    file_long_name = long_name[-1]
    file_name = file_long_name.split(".")[0]
    raster_file = path+"/Overlay/" + file_name+"_Co.tif"
    shp_file = Points[k]
    output_shp = r"F:\Water_Quality\KDE_SEP_TIF_All\0629\Points\ExtractValue/"+file_long_name
    ExtractValuesToPoints(shp_file, raster_file, output_shp,
                      "NONE", "ALL")

In [126]:
Points_with_value = glob.glob(os.path.join(str(r"F:\Water_Quality\KDE_SEP_TIF_All\0629\Points\ExtractValue") , "*.shp"))

In [128]:
all_points_pd = pd.DataFrame()
for i in range(len(Points_with_value)):
    gdf = gpd.read_file(Points_with_value[i])
    listcolumns = gdf.columns.tolist()
    gdf = gdf.rename(columns={listcolumns[0]:'AreaID',
                              listcolumns[1]:'ManagedAreaName',
                              listcolumns[2]:'ParameterName',
                              listcolumns[3]:'ProgramID',
                              listcolumns[4]:'ProgramName',
                              listcolumns[5]:'Type',
                              listcolumns[6]:'ProgramLocation',
                              listcolumns[7]:'Lat',
                              listcolumns[8]:'Long',
                              listcolumns[9]:'RASTERVALU',
                              listcolumns[10]:'SEP',
                              listcolumns[11]:'KDE',
                              listcolumns[12]:'geometry'})
#     gdf = gdf.drop('RASTERVALU',axis=1)
    gdf["SEP"] = gdf["SEP"].replace([0,1,2,3],["None","Low","Median","High"])
    gdf["KDE"] = gdf["KDE"].replace([0,1,2,3],["None","Low","Median","High"])
    print(len(gdf))
    all_points_pd = pd.concat([all_points_pd,gdf])
display(all_points_pd)
#     list_implication = []
#     list_solution = []
#     for j in range(len(gdf)):
#         gdf_row = gdf[j:j+1]
#         SEP = str(gdf_row.SEP)
#         KDE = str(gdf_row.KDE)
#         if (SEP in "High") & (KDE in "High"):
#             print("a")
#             list_implication.insert(j,"Natural variation, is it a seasonal issue, might be unexplained variation")
#             list_solution.insert(j," ")
#         elif (SEP == "Low") & (KDE == "High"):
#             list_implication.insert(j,"Potential redundancy")
#             list_solution.insert(j,"Identify points")
#         elif (SEP == "Low") & (KDE == "Low"):
#             list_implication.insert(j,"No change needed / low priority")
#             list_solution.insert(j," ")
#         elif (SEP == "High") & (KDE == "Low"):
#             list_implication.insert(j,"Potential need for more stations")
#             list_solution.insert(j,"Identify areas")
#         else:
#             list_implication.insert(j," ")
#             list_solution.insert(j," ")
#     print(list_implication)
#     print(list_solution)
        

236
184
31
203
191
3248
3190
2949
171
346
826
32
507
551
2803
2604
2492
865
888
182
162
87
70


,AreaID,ManagedAreaName,ParameterName,ProgramID,ProgramName,Type,ProgramLocation,Lat,Long,RASTERVALU,SEP,KDE,geometry
0,6,Biscayne Bay Aquatic Preserve,Dissolved Oxygen,4058,City of Miami Beach Water Monitoring,Discrete,25,25.794300,-80.155300,5,Low,High,POINT (784940.913 204924.563)
1,6,Biscayne Bay Aquatic Preserve,Dissolved Oxygen,5002,Florida STORET / WIN,Discrete,WIN_21FLDADE_LR01,25.844860,-80.173851,4,Median,High,POINT (782910.144 210474.060)
2,6,Biscayne Bay Aquatic Preserve,Dissolved Oxygen,103,EPA STOrage and RETrieval Data Warehouse (STORET),Discrete,21FLDADE_WQX-BB47,25.336794,-80.320077,9,Low,Low,POINT (769962.429 153690.811)
3,6,Biscayne Bay Aquatic Preserve,Dissolved Oxygen,5026,North Biscayne Bay Seagrass Loss Water Quality...,Discrete,WQ05,25.808136,-80.140808,5,Low,High,POINT (786343.378 206504.300)
4,6,Biscayne Bay Aquatic Preserve,Dissolved Oxygen,4058,City of Miami Beach Water Monitoring,Discrete,18,25.807500,-80.125100,-9999,None,None,POINT (787917.603 206483.081)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,5002,Florida STORET / WIN,Discrete,WIN_21FLA_27010079,29.722944,-81.264222,7,Low,High,POINT (664258.441 638054.603)
66,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmpcwq,29.667071,-81.257403,7,Low,High,POINT (665054.955 631867.561)
67,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmpiwq,30.050857,-81.367465,7,Low,High,POINT (653510.323 674231.938)
68,20,Guana Tolomato Matanzas National Estuarine Res...,Turbidity,4054,Guana Tolomato Matanzas National Estuarine Res...,Continuous,gtmsswq,29.868851,-81.307428,7,Low,High,POINT (659732.033 654157.227)


In [129]:
all_points_gpd = gpd.GeoDataFrame(all_points_pd,geometry="geometry")

In [130]:
all_points_gpd.to_file(r"F:\Water_Quality\KDE_SEP_TIF_All\0629\Points\Result\all_points.shp")

C:\Users\xiang11\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [21]:
for i in listPara:
    para = dictPara[i]
    data_export = data_gdf[data_gdf["ParameterName"]==i]
    data_export.to_file(r"F:\Water_Quality\KDE_SEP_TIF_All\Points\new/"+para+".shp")

C:\Users\xiang11\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  after removing the cwd from sys.path.


Next step is Finishied by Arcgis Pro